# Exercises for Introduction to Quantum Computing

Name: Pugazharasu Anancia Devaneyan (s6puanan) <br />
Matriculation number: 3300280

In [11]:
#Importing the required libraries
import matplotlib.pyplot as plt
import math
import numpy as np
from math import pi
from qiskit import IBMQ, BasicAer
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import QFT
from scipy import signal

# 1 Quantum Simulation on Quantum Hardware

a)

b)

c)

In [ ]:
from qiskit_ibm_provider import IBMProvider
IBMProvider.save_account(token=MY_API_TOKEN)

In [ ]:
p r o v i d e r = IBMProvider ( )
2
 d e v i c e l i s t = p r o v i d e r . backends ( )
3 f o r dev i n
 device list :
4
 p r i n t ( dev . name + ’ : ’ + s t r ( dev . c o n f i g u r a t i o n ( ) . n q u b i t s ) +’q u b i t s ’ )

In [ ]:
 num shots hardware = 1024
2
 hardware backend = p r o v i d e r . g e t b a c k e n d ( ’ i b m q q u i t o ’ )
3 j o b = e x e c u t e ( qc , backend=hardware backend , s h o t s=num shots hardware )

In [ ]:
from q i s k i t . p r o v i d e r s . f a k e p r o v i d e r import FakeQuitoV2
2
 hardware backend = FakeQuitoV2 ( )
3 j o b = e x e c u t e ( qc , backend=hardware backend , s h o t s=num shots hardware )

# 2 Pauli Measurements

a)

b)